In [117]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from tqdm import tqdm
from transformers import AutoTokenizer

In [118]:
train_path = '../../data/train.csv'
valid_path = '../../data/dev.csv'
model_name = 'monologg/koelectra-base-v3-discriminator'

In [119]:
train_data = pd.read_csv(train_path)
valid_data = pd.read_csv(valid_path)

In [120]:
train_data.sample(10)

,id,source,sentence_1,sentence_2,label,binary-label
9013,boostcamp-sts-v1-train-9013,slack-sampled,ㅋㅋ 나중에 실험해보시구 알려주세요!,일단 신청해보시고 알려주세요!!,1.0,0.0
5678,boostcamp-sts-v1-train-5678,petition-sampled,사내하청을 직접고용 법제화 해주세요,인도 안쪽 보행' 을 의무법제화 해주십시오.,0.2,0.0
1192,boostcamp-sts-v1-train-1192,nsmc-sampled,니콜라스케이지의 눈빛이 너무나 슬프고 마음이 아프다..,니콜라스 케이지 역시... 재밌게 잘 봤습니다,2.0,0.0
6017,boostcamp-sts-v1-train-6017,nsmc-rtt,찍소리라도 내야겠다는 마인드가 졸라 멋짐,삐딱하게 해야 한다는 마음가짐이 있어서 멋있다.,1.8,0.0
245,boostcamp-sts-v1-train-245,nsmc-rtt,"스토리정말유치하다,","이야기는 유치하고,",4.2,1.0
2792,boostcamp-sts-v1-train-2792,petition-sampled,음주운전 처벌을 강화해주세요,음주운전 처벌을 강화해주세요!,4.4,1.0
4831,boostcamp-sts-v1-train-4831,petition-sampled,20만 화물자동차 운송료,화물운송 사업에 관한 청원,0.4,0.0
8466,boostcamp-sts-v1-train-8466,slack-rtt,허먼밀러 헤드레스트 도착!,허먼밀러 헤드레스트가 도착했습니다!,4.2,1.0
6481,boostcamp-sts-v1-train-6481,nsmc-sampled,플롯의 구성도 베스트,제로 다크서티보다 완성도 떨어지는 영화,0.0,0.0
4887,boostcamp-sts-v1-train-4887,petition-sampled,스윙스의 의가사 제대와 랩과외로 인한 수익활동!?,"모든 백신(예방주사)의 성분분석 및 부작용 사례 전수조사,",0.0,0.0


In [121]:
valid_data.sample(10)

,id,source,sentence_1,sentence_2,label,binary-label
338,boostcamp-sts-v1-dev-338,petition-rtt,대통령님 제 2의 촛불의 희생자는 당신이 될수도있습니다,"대통령, 당신은 두 번째 촛불의 희생자가 될 수 있습니다",4.4,1.0
55,boostcamp-sts-v1-dev-055,nsmc-sampled,만들어진 년도를 감안해도 정말 살떨리는 유지판타지 주인공들의 연기력역시 바닥ㅋㅋ 어...,진짜 배우들연기력이 대박이다..,0.4,0.0
417,boostcamp-sts-v1-dev-417,slack-rtt,다음에 기회가 된다면 만나서 밥이라도 먹기루..!,다음에 기회가 된다면 만나서 밥먹자..!,4.2,1.0
342,boostcamp-sts-v1-dev-342,petition-sampled,피의자 김성수 강력한 처벌 원합니다,강서구 pc방 살인사건 피의자 김성수 감형 반대,2.8,1.0
258,boostcamp-sts-v1-dev-258,petition-sampled,간통죄 부활시켜주세요,간통죄 부활해주세요!!,4.0,1.0
130,boostcamp-sts-v1-dev-130,petition-sampled,국회의원 월급 삭감,국회의원 급여 미지급,1.8,0.0
75,boostcamp-sts-v1-dev-075,petition-rtt,위대하신 문대통령님!,위대한 문재인 대통령!,4.8,1.0
519,boostcamp-sts-v1-dev-519,slack-sampled,넵 저희는 github에서 관리중인데 사용자분들은 편의상 저기에 받았습니다.,대문자는 shift 눌러서 사용하고 있습니다.,0.0,0.0
401,boostcamp-sts-v1-dev-401,petition-sampled,수능성적으로 한줄세우기 해야,단순하게 수능으로 한줄 세우기 해야 합니다.,3.8,1.0
445,boostcamp-sts-v1-dev-445,slack-sampled,특히 오프닝 크레딧이 있는 시퀀스가 최고라고 느꼈는데요.,"OST도 매우 좋아서, 그 오프닝 크레딧에 있는 음악을 올려봅니다.",1.6,0.0


In [122]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_tokens(["<PERSON>"])
def tokenizing_text_id(text):
    return tokenizer(text)['input_ids']

def tokenizing_text(text):
    return tokenizer.tokenize(text)
    

In [123]:
def preprocessing_df(df):
    df['sentence_1_tokens_id'] = df['sentence_1'].apply(tokenizing_text_id)
    df['sentence_1_tokens'] = df['sentence_1'].apply(tokenizing_text)
    df['sentence_2_tokens_id'] = df['sentence_2'].apply(tokenizing_text_id)
    df['sentence_2_tokens'] = df['sentence_2'].apply(tokenizing_text)
    return df
    


In [124]:
tokenizer.add_tokens('##휏')

1

In [125]:
train_data = preprocessing_df(train_data)

In [116]:
train_data.sample(10)

,id,source,sentence_1,sentence_2,label,binary-label,sentence_1_tokens_id,sentence_1_tokens,sentence_2_tokens_id,sentence_2_tokens
8501,boostcamp-sts-v1-train-8501,slack-sampled,"오늘 하루 행복한 시간 보내시고, 휴일도 즐겁게 보내셨으면 좋겠습니다!!",오늘 하루 잘 보내시고 즐거운 주말 맞이하세요!,3.4,1.0,"[2, 6451, 6674, 7003, 4283, 6251, 6823, 25023,...","[오늘, 하루, 행복, ##한, 시간, 보내, ##시고, ,, 휴일, ##도, 즐겁...","[2, 6451, 6674, 3258, 6823, 25023, 11357, 7856...","[오늘, 하루, 잘, 보내, ##시고, 즐거운, 주말, 맞이, ##하, ##세요, !]"
6664,boostcamp-sts-v1-train-6664,petition-rtt,"가짜간첩 피해자들에 대한 보상과 당시 고문에 관련했던 가해자(법조인, 정치인 등)들...","가짜 간첩 피해자에 대한 배상과 당시 고문에 가담한 가해자(법조인, 정치인 등)에 ...",4.6,1.0,"[2, 10721, 4629, 4853, 7657, 4006, 4073, 6230,...","[가짜, ##간, ##첩, 피해자, ##들, ##에, 대한, 보상, ##과, 당시,...","[2, 10721, 16139, 7657, 4073, 6230, 9405, 4047...","[가짜, 간첩, 피해자, ##에, 대한, 배상, ##과, 당시, 고문, ##에, 가..."
2941,boostcamp-sts-v1-train-2941,petition-rtt,부정부폐비리의천국 전정부 인전못합니다.[펌옴],부패와 부패의 낙원,1.4,0.0,"[2, 7323, 4258, 4719, 4164, 4108, 4234, 4156, ...","[부정, ##부, ##폐, ##비, ##리, ##의, ##천, ##국, 전, ##정...","[2, 10298, 4192, 10298, 4234, 23587, 3]","[부패, ##와, 부패, ##의, 낙원]"
555,boostcamp-sts-v1-train-555,slack-sampled,부트캠프이 최고의 AI 선생님이라고 말씀해주셨던 <PERSON> 님,부트캠프 멘토님께서 공유해주신 짤입니다,0.6,0.0,"[2, 2799, 4039, 33897, 4007, 6557, 4234, 9760,...","[부, ##트, ##캠프, ##이, 최고, ##의, AI, 선생, ##님, ##이,...","[2, 2799, 4039, 33897, 13075, 4660, 5253, 4129...","[부, ##트, ##캠프, 멘토, ##님, ##께, ##서, 공유, ##해, ##주..."
7162,boostcamp-sts-v1-train-7162,petition-rtt,지지율 올릴방법 한진운수 대한항공 처벌,"승인등급 높이는 방법 한진운송, 대한항공 처벌",3.4,1.0,"[2, 9706, 11928, 4212, 4472, 11971, 4231, 4141...","[지지율, 올릴, ##방, ##법, 한진, ##운, ##수, 대한항공, 처벌]","[2, 8305, 4291, 4027, 7076, 4034, 6472, 11971,...","[승인, ##등, ##급, 높이, ##는, 방법, 한진, ##운, ##송, ,, 대..."
8725,boostcamp-sts-v1-train-8725,nsmc-sampled,수술할정도로 아파본사람들만이 제대로 동감할수 있는영화..,나이가 좀 들은 사람이 보면 공감할 내용....,1.6,0.0,"[2, 7567, 4519, 4142, 8766, 6735, 4372, 15794,...","[수술, ##할, ##정, ##도로, 아파, ##본, ##사람, ##들, ##만, ...","[2, 7050, 4070, 3305, 2441, 4112, 6226, 4007, ...","[나이, ##가, 좀, 들, ##은, 사람, ##이, 보, ##면, 공감, ##할,..."
3798,boostcamp-sts-v1-train-3798,slack-rtt,넘 후다닥 간거 같아 아쉽네여 ㅜㅜㅜ,너무 뒤처져서 아쉽네요.,0.8,0.0,"[2, 2273, 3826, 28071, 2012, 4216, 2024, 4153,...","[넘, 후, ##다닥, 간, ##거, 같, ##아, 아쉽, ##네, ##여, ㅜㅜㅜ]","[2, 6395, 20982, 11789, 10969, 4116, 4150, 18, 3]","[너무, 뒤처, ##져서, 아쉽, ##네, ##요, .]"
2036,boostcamp-sts-v1-train-2036,petition-sampled,교육공무직을 없애주세요!,관공서 업무시간 늘려주세요!,0.0,0.0,"[2, 6295, 4333, 4185, 4497, 4292, 9347, 4076, ...","[교육, ##공, ##무, ##직, ##을, 없애, ##주, ##세요, !]","[2, 25124, 6805, 12210, 11532, 4076, 8553, 5, 3]","[관공서, 업무, ##시간, 늘려, ##주, ##세요, !]"
7763,boostcamp-sts-v1-train-7763,nsmc-sampled,감독이름믿고 봤습니다. 역시 재밌네요. 마음이 따뜻해지는 영화였습니다,정말 잘만든영화 . 참 재밌게봤습니다.,2.4,0.0,"[2, 6543, 4007, 4424, 5444, 4219, 2791, 4576, ...","[감독, ##이, ##름, ##믿, ##고, 봤, ##습, ##니다, ., 역시, ...","[2, 6595, 3258, 4172, 4373, 22682, 18, 3425, 1...","[정말, 잘, ##만, ##든, ##영화, ., 참, 재밌, ##게, ##봤, ##..."
8767,boostcamp-sts-v1-train-8767,nsmc-sampled,순수하고 감동적이었던...,진짜 감동적이더라....,2.2,0.0,"[2, 9115, 4279, 4219, 9186, 4199, 4007, 4480, ...","[순수, ##하, ##고, 감동, ##적, ##이, ##었, ##던, ., ., .]","[2, 7082, 9186, 4199, 4007, 26093, 18, 18, 18,...","[진짜, 감동, ##적, ##이, ##더라, ., ., ., .]"


In [108]:
tokenizer('딿뚫너무좋아')



1

In [109]:
tokenizer.tokenize('딿뚫너무좋아 안녕 나는 행복해 딿뚫').count('[UNK]')

2

In [110]:
cnt = 0
for item in train_data['sentence_1_tokens']:
    cnt += item.count('[UNK]')
print(cnt)
for item in train_data['sentence_2_tokens']:
    cnt += item.count('[UNK]')
cnt

66


118

In [103]:
cnt

118

AttributeError: 'ElectraTokenizerFast' object has no attribute 'return_token_type_ids'